In [1]:
import sqlite3
import numpy as np
import pandas as pd

Переведем данные из экселевского в массив NumPy.
Я переименовал названия столбцов, для более удобной работы.
И создал отдельный файл, в который коротеньким скриптом на VBA добавил только "плохие" кредиты.

In [2]:
df_main = pd.read_excel("data.xlsx")
df_main.head()

,id,type,cr1,cr2,cr3,cl1,cl2,cl3,cl4,cl5,cl6,cl7,flag
0,1,2,10,5,5,1,2,0,7,1,12,16,0
1,2,7,3,1,6,2,3,1,7,4,10,15,0
2,3,7,10,1,6,2,4,1,7,4,11,20,0
3,4,7,4,1,10,1,7,1,7,4,8,19,0
4,5,7,6,1,10,2,6,1,7,4,8,3,0


In [3]:
arr_main = df_main.values
arr_main.shape

(74405, 13)

In [4]:
df_ans = pd.read_excel("dataAns.xlsx")
df_ans.head()

,id,type,cr1,cr2,cr3,cl1,cl2,cl3,cl4,cl5,cl6,cl7,flag
0,535,7,5,4,10,2,7,0,7,1,14,1,1
1,711,7,4,1,10,2,7,1,7,1,13,14,1
2,793,7,2,1,6,2,2,0,4,2,12,13,1
3,913,7,5,5,10,1,6,0,4,1,10,17,1
4,1036,7,5,5,10,1,3,0,4,1,9,14,1


In [5]:
arr_ans = df_ans.values
arr_ans.shape

(1154, 13)

Создадим таблицу с помощью sqlite3, чтобы можно было быстро проверять результат.

In [6]:
def sql_query(SQL_QUERY, db):
    connection = sqlite3.connect(db)
    cursor = connection.cursor()
    try:
        if str.lower(SQL_QUERY)[:6] == 'select':
            df = pd.read_sql_query(SQL_QUERY, connection)
            return df

        sql_command = SQL_QUERY
        cursor.execute(sql_command)
    finally:
        connection.commit()
        connection.close()

In [7]:
sql_command = """ CREATE TABLE credit (
id INTEGER PRIMARY KEY,
type INTEGER, cr1 INTEGER, cr2 INTEGER, cr3 INTEGER,
cl1 INTEGER, cl2 INTEGER, cl3 INTEGER, cl4 INTEGER,
cl5 INTEGER, cl6 INTEGER, cl7 INTEGER, flag INTEGER
);
"""
      
sql_query(sql_command, "data_.db")

In [8]:
for el in arr_main:
    format_str = """INSERT INTO credit (id, type, cr1, cr2, cr3, cl1, cl2, cl3, cl4, cl5, cl6, cl7, flag)
    VALUES ({id_} , {type_}, {cr1}, {cr2}, {cr3}, {cl1}, {cl2}, {cl3}, {cl4}, {cl5}, {cl6}, {cl7}, {flag})"""
    sql_com = format_str.format(id_=el[0],type_=el[1], cr1=el[2], cr2=el[3], cr3=el[4], cl1=el[5],
                                cl2=el[6], cl3=el[7], cl4=el[8], cl5=el[9], cl6=el[10], cl7=el[11], flag=el[12])
    sql_query(sql_com, "data_.db")

Теперь создадим частотные словари для каждого признака изначального датасета и датасета с "плохими" кредитами и посмотрим, какие значения для каждого признака встречаются в "плохих" кредитах.

In [9]:
d_main = dict() #type
for el in arr_main:
    d_main[el[1]] = d_main.get(el[1], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 147 | 2 : 16893 | 3 : 3417 | 4 : 8101 | 5 : 2327 | 6 : 2021 | 7 : 40470 | 8 : 939 | 9 : 33 | 10 : 57 | 

In [10]:
d_ans = dict() #type
for el in arr_ans:
    d_ans[el[1]] = d_ans.get(el[1], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

2 : 80 | 5 : 13 | 7 : 1060 | 10 : 1 | 

In [11]:
d_main = dict() #cr1
for el in arr_main:
    d_main[el[2]] = d_main.get(el[2], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 763 | 2 : 5222 | 3 : 7036 | 4 : 7827 | 5 : 6676 | 6 : 6400 | 7 : 6740 | 8 : 5606 | 9 : 3724 | 10 : 4940 | 11 : 3069 | 12 : 2403 | 13 : 1669 | 14 : 1402 | 15 : 1248 | 16 : 1153 | 17 : 942 | 18 : 701 | 19 : 1773 | 20 : 912 | 21 : 500 | 22 : 394 | 23 : 356 | 24 : 354 | 25 : 248 | 26 : 173 | 27 : 145 | 28 : 268 | 29 : 115 | 30 : 1646 | 

In [12]:
d_ans = dict() #cr1
for el in arr_ans:
    d_ans[el[2]] = d_ans.get(el[2], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

2 : 6 | 3 : 5 | 4 : 16 | 5 : 118 | 6 : 198 | 7 : 254 | 8 : 210 | 9 : 130 | 10 : 114 | 11 : 50 | 12 : 21 | 13 : 12 | 14 : 5 | 15 : 5 | 16 : 2 | 17 : 2 | 18 : 2 | 19 : 1 | 20 : 1 | 28 : 1 | 30 : 1 | 

In [13]:
d_main = dict() #cr2
for el in arr_main:
    d_main[el[3]] = d_main.get(el[3], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 44493 | 2 : 1496 | 3 : 9381 | 4 : 5946 | 5 : 4944 | 6 : 3265 | 7 : 1806 | 8 : 1096 | 9 : 621 | 10 : 392 | 11 : 639 | 12 : 161 | 13 : 70 | 14 : 27 | 15 : 44 | 16 : 8 | 17 : 10 | 18 : 3 | 19 : 3 | 

In [14]:
d_ans = dict() #cr2
for el in arr_ans:
    d_ans[el[3]] = d_ans.get(el[3], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

1 : 415 | 2 : 3 | 3 : 151 | 4 : 253 | 5 : 224 | 6 : 82 | 7 : 23 | 8 : 3 | 

In [15]:
d_main = dict() #cr3
for el in arr_main:
    d_main[el[4]] = d_main.get(el[4], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 10 | 2 : 79 | 3 : 172 | 4 : 4681 | 5 : 4623 | 6 : 13071 | 7 : 4192 | 8 : 103 | 9 : 4859 | 10 : 34417 | 11 : 1380 | 12 : 2377 | 13 : 21 | 14 : 2 | 15 : 2770 | 16 : 17 | 17 : 2 | 18 : 282 | 19 : 50 | 20 : 389 | 21 : 82 | 22 : 78 | 23 : 685 | 24 : 63 | 

In [16]:
d_ans = dict() #cr3
for el in arr_ans:
    d_ans[el[4]] = d_ans.get(el[4], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

4 : 14 | 5 : 15 | 6 : 319 | 7 : 2 | 10 : 764 | 12 : 13 | 23 : 26 | 24 : 1 | 

In [17]:
d_main = dict() #cl1
for el in arr_main:
    d_main[el[5]] = d_main.get(el[5], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 35076 | 2 : 39329 | 

In [18]:
d_ans = dict() #cl1
for el in arr_ans:
    d_ans[el[5]] = d_ans.get(el[5], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

1 : 321 | 2 : 833 | 

In [19]:
d_main = dict() #cl2
for el in arr_main:
    d_main[el[6]] = d_main.get(el[6], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 1889 | 2 : 12507 | 3 : 15657 | 4 : 12476 | 5 : 9219 | 6 : 6754 | 7 : 4839 | 8 : 4365 | 9 : 3152 | 10 : 1998 | 11 : 1265 | 12 : 229 | 13 : 54 | 14 : 1 | 

In [20]:
d_ans = dict() #cl2
for el in arr_ans:
    d_ans[el[6]] = d_ans.get(el[6], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

2 : 69 | 3 : 340 | 4 : 314 | 5 : 157 | 6 : 117 | 7 : 82 | 8 : 64 | 9 : 11 | 

In [21]:
d_main = dict() #cl3
for el in arr_main:
    d_main[el[7]] = d_main.get(el[7], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

0 : 52097 | 1 : 22308 | 

In [22]:
d_ans = dict() #cl3
for el in arr_ans:
    d_ans[el[7]] = d_ans.get(el[7], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

0 : 776 | 1 : 378 | 

In [23]:
d_main = dict() #cl4
for el in arr_main:
    d_main[el[8]] = d_main.get(el[8], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 3 | 2 : 5412 | 3 : 15 | 4 : 28447 | 5 : 3625 | 6 : 1005 | 7 : 35888 | 8 : 10 | 

In [24]:
d_ans = dict() #cl4
for el in arr_ans:
    d_ans[el[8]] = d_ans.get(el[8], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

2 : 33 | 4 : 400 | 7 : 721 | 

In [25]:
d_main = dict() #cl5
for el in arr_main:
    d_main[el[9]] = d_main.get(el[9], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 35828 | 2 : 7547 | 3 : 362 | 4 : 30536 | 5 : 132 | 

In [26]:
d_ans = dict() #cl5
for el in arr_ans:
    d_ans[el[9]] = d_ans.get(el[9], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

1 : 774 | 2 : 29 | 3 : 3 | 4 : 348 | 

In [27]:
d_main = dict() #cl6
for el in arr_main:
    d_main[el[10]] = d_main.get(el[10], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 5 | 2 : 113 | 3 : 143 | 4 : 1259 | 5 : 2055 | 6 : 3097 | 7 : 6922 | 8 : 7213 | 9 : 9697 | 10 : 8417 | 11 : 9817 | 12 : 3414 | 13 : 6285 | 14 : 2288 | 15 : 3561 | 16 : 1244 | 17 : 2476 | 18 : 664 | 19 : 1220 | 20 : 272 | 21 : 1804 | 22 : 84 | 23 : 170 | 24 : 59 | 25 : 517 | 26 : 38 | 27 : 159 | 28 : 43 | 29 : 96 | 30 : 18 | 31 : 1255 | 

In [28]:
d_ans = dict() #cl6
for el in arr_ans:
    d_ans[el[10]] = d_ans.get(el[10], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

2 : 3 | 7 : 2 | 8 : 11 | 9 : 35 | 10 : 230 | 11 : 324 | 12 : 170 | 13 : 248 | 14 : 82 | 15 : 20 | 16 : 12 | 17 : 10 | 19 : 2 | 20 : 2 | 21 : 2 | 31 : 1 | 

In [29]:
d_main = dict() #cl7
for el in arr_main:
    d_main[el[11]] = d_main.get(el[1], 0) + 1
for k in sorted(d_main.keys()):
    print (k, ':', d_main[k], end=' | ')

1 : 1744 | 2 : 1742 | 3 : 1744 | 4 : 1740 | 5 : 1743 | 6 : 1744 | 7 : 1744 | 8 : 1741 | 9 : 1740 | 10 : 1741 | 11 : 1744 | 12 : 1741 | 13 : 1743 | 14 : 1745 | 15 : 1742 | 16 : 1741 | 17 : 1740 | 18 : 1743 | 19 : 1743 | 20 : 1743 | 21 : 1738 | 

In [30]:
d_ans = dict() #cl7
for el in arr_ans:
    d_ans[el[11]] = d_ans.get(el[11], 0) + 1
for k in sorted(d_ans.keys()):
    print (k, ':', d_ans[k], end=' | ')

1 : 235 | 2 : 22 | 3 : 16 | 4 : 28 | 5 : 22 | 6 : 33 | 7 : 37 | 8 : 16 | 9 : 43 | 10 : 41 | 11 : 61 | 12 : 58 | 13 : 73 | 14 : 82 | 15 : 84 | 16 : 89 | 17 : 83 | 18 : 80 | 19 : 43 | 20 : 8 | 

Например из признака "Тип точки продаж" мы возьмем только 2 и 7 тип, так для других типов количество "плохих" кредитов либо равно 0, либо очень мало. Признаки "Клиент 1" и "Клиент 3" вообще не нужно рассматривать, так как они биномиальные и принимают оба значения. И т.д., так я проанализировал все признакми, и проверил несколько предполагаемых ответов, из которых оставил на мой взгляд лучший.

In [31]:
sql_command = """SELECT COUNT(id) AS cnt_id FROM credit WHERE type IN (2, 7) AND cr1 IN (5, 6, 7, 8, 9, 10, 11, 12, 13)
AND cr2 IN (1, 3, 4, 5, 6, 7) AND cr3 IN (5, 6, 10, 23) AND cl2 IN (2, 3, 4, 5, 6, 7, 8, 9) AND cl4 IN (2, 4, 7)
AND cl5 IN (1, 2, 4) AND cl6 IN (10, 11, 12, 13, 14, 15, 16)
AND cl7 IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)
UNION
SELECT COUNT(id) AS cnt_id FROM credit WHERE type IN (2, 7) AND cr1 IN (5, 6, 7, 8, 9, 10, 11, 12, 13)
AND cr2 IN (1, 3, 4, 5, 6, 7) AND cr3 IN (5, 6, 10, 23) AND cl2 IN (2, 3, 4, 5, 6, 7, 8, 9) AND cl4 IN (2, 4, 7)
AND cl5 IN (1, 2, 4) AND cl6 IN (10, 11, 12, 13, 14, 15, 16)
AND cl7 IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19) AND flag = 1"""
sql_query(sql_command, "data_.db") 

,cnt_id
0,1015
1,14610


В итоге мой сегмент состоит из 14610 элементов, это лишь 1/6 всего датасэта, и содержит в себе 1015 "плохих" кредитов. Конечно количество таких кредитов можно было бы несложно увеличить, добавив еще некоторые значения, но тогда сильно возрастет общее количество элементов. Я подумал, что такой результат будет "золотой серединой"

**И собственно мой ответ:**

Тип точки продаж in [2, 7] AND Кредит 1 in [5, 6, 7, 8, 10, 11, 12, 13] AND Кредит 2 in [1, 3, 4, 5, 6, 7]

AND Кредит 3 in [5, 6, 10, 23] AND Клиент 2 in [2, 3, 4, 5, 6, 7, 8, 9] AND Клиент 4 in [2, 4, 7]

AND Клиент 5 in [1, 2, 4] AND Клиент 6 in [10, 11, 12, 13, 14, 15, 16]

AND Клиент 7 in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]